In [1]:
import pandas as pd
from tensorflow import keras
from os import path
import pickle


In [2]:
from proteinbert.finetuning import encode_train_and_valid_sets, encode_dataset
from proteinbert import OutputType, OutputSpec, evaluate_by_len, load_pretrained_model

In [3]:
from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, \
finetune, evaluate_by_len

from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

In [4]:
import wandb
from wandb.keras import WandbCallback

In [5]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [6]:
DATA_DIR = "../../data/"

In [7]:
OUTPUT_TYPE = OutputType(False, 'binary')
UNIQUE_LABELS = [0, 1]
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

In [8]:
pretrained_model_generator, input_encoder = load_pretrained_model("../../data/protein_bert/", "epoch_92400_sample_23500000.pkl")

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train_set = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_train.csv"), index_col=0)
test_set = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_test.csv"), index_col=0)
train_set["seq"] = train_set["heavy"] + train_set["light"]
test_set["seq"] = test_set["heavy"] + test_set["light"]

In [11]:
len(train_set)

1291

In [12]:
len(test_set)

260

In [13]:
learning_rate = 1e-4
patience = (6, 4)

In [14]:
sizes = [0.5, 0.6, 0.7, 0.8, 0.9]

In [16]:
def train_and_save_model(train_data, valid_data, test_data, size):
    wandb.init(project=f"Dataset size exp", entity="kvetab")
    model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
            get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

    training_callbacks = [
        keras.callbacks.ReduceLROnPlateau(patience = patience[1], factor = 0.25, min_lr = 1e-07, verbose = 1),
        keras.callbacks.EarlyStopping(patience = patience[0], restore_best_weights = True),
        WandbCallback()
    ]

    epoch_num = 100
    batch_size = 128
    #learning_rate = 1e-5
    wandb.config = {
          "learning_rate": learning_rate,
          "epochs": epoch_num * 2,
          "batch_size": batch_size
        }
    finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data["seq"], train_data["Y"], valid_data['seq'], valid_data["Y"], \
            seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = learning_rate, begin_with_frozen_pretrained_layers = True, \
            lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = learning_rate / 10, callbacks = training_callbacks)
    mod = model_generator.create_model(seq_len = 512)
    mod_name = f"2022_04_22_size{size}"
    mod.save(path.join(DATA_DIR, f"protein_bert/by_data_size/{mod_name}"))

    results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
            start_seq_len = 512, start_batch_size = 32)
    fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
    f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
    return confusion_matrix, f1

In [15]:
cms = {}
f1s = {}

In [17]:
for size in sizes:
    train, valid = train_test_split(train_set, test_size=1-size, random_state=42, stratify=train_set["Y"])
    #test = pd.concat([test, test_set])
    test = test_set
    #valid, test = train_test_split(test, test_size=0.5, random_state=333, stratify=test["Y"])
    print(len(train), len(valid), len(test))
    cm, f1_score = train_and_save_model(train, valid, test, size)
    cms[size] = cm
    f1s[size] = f1_score

645 646 260


wandb: Currently logged in as: kvetab (use `wandb login --relogin` to force relogin)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
wandb: wandb version 0.12.15 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


[2022_04_24-16:50:26] Training set: Filtered out 0 of 645 (0.0%) records of lengths exceeding 510.
[2022_04_24-16:50:27] Validation set: Filtered out 0 of 646 (0.0%) records of lengths exceeding 510.
[2022_04_24-16:50:27] Training with frozen pretrained layers...


2022-04-24 16:50:27.159831: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-24 16:50:27.728399: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9656 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:41:00.0, compute capability: 7.5
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-04-24 16:50:29.565892: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are en

Epoch 1/100


...

Epoch 2/100
10/10 [==============================] - 1s 110ms/step - loss: 0.5500 - val_loss: 0.5621
Epoch 3/100
10/10 [==============================] - 1s 109ms/step - loss: 0.4897 - val_loss: 0.4898
Epoch 4/100
10/10 [==============================] - 1s 111ms/step - loss: 0.4600 - val_loss: 0.4561
Epoch 5/100
10/10 [==============================] - 1s 108ms/step - loss: 0.4398 - val_loss: 0.4398
Epoch 6/100
10/10 [==============================] - 1s 111ms/step - loss: 0.4347 - val_loss: 0.4205
Epoch 7/100
10/10 [==============================] - 1s 112ms/step - loss: 0.4260 - val_loss: 0.4178
Epoch 8/100
10/10 [==============================] - 1s 111ms/step - loss: 0.4149 - val_loss: 0.4127
Epoch 9/100
10/10 [==============================] - 1s 113ms/step - loss: 0.4075 - val_loss: 0.4772
Epoch 10/100
10/10 [==============================] - 1s 107ms/step - loss: 0.4315 - val_loss: 0.4717
Epoch 11/100
10/10 [==============================] - 1s 108ms/step - loss: 0.4353 - val_l

10/10 [==============================] - 10s 465ms/step - loss: 0.3663 - val_loss: 0.3977
Epoch 2/100
10/10 [==============================] - 2s 248ms/step - loss: 0.3775 - val_loss: 0.3766
Epoch 3/100
10/10 [==============================] - 2s 248ms/step - loss: 0.3638 - val_loss: 0.3760
Epoch 4/100
10/10 [==============================] - 2s 248ms/step - loss: 0.3658 - val_loss: 0.3703
Epoch 5/100
10/10 [==============================] - 3s 249ms/step - loss: 0.3539 - val_loss: 0.4184
Epoch 6/100
10/10 [==============================] - 2s 248ms/step - loss: 0.3805 - val_loss: 0.3657
Epoch 7/100
10/10 [==============================] - 3s 253ms/step - loss: 0.3624 - val_loss: 0.3786
Epoch 8/100
10/10 [==============================] - 2s 248ms/step - loss: 0.3466 - val_loss: 0.3756
Epoch 9/100
10/10 [==============================] - 3s 252ms/step - loss: 0.3505 - val_loss: 0.3670
Epoch 10/100
10/10 [==============================] - 3s 253ms/step - loss: 0.3324 - val_loss: 0.3572


19/19 [==============================] - 13s 377ms/step - loss: 0.2923 - val_loss: 0.3448


INFO:tensorflow:Assets written to: ../../data/protein_bert/by_data_size/2022_04_22_size0.9/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/by_data_size/2022_04_22_size0.9/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [18]:
size = 1
train = train_set
valid, test = train_test_split(test_set, test_size=0.5, random_state=333, stratify=test_set["Y"])
print(len(train_set), len(valid), len(test))
cm, f1_score = train_and_save_model(train, valid, test, size)
cms[size] = cm
f1s[size] = f1_score

1291 130 130


epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██▁▂▂▂▂▃▃▃▃▄▄▄▅▅▁
loss,█▅▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▁▁▂
lr,████████▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▄▃▃▄▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▂▂▂▁▂▁▁▁▂▂▂▁
best_epoch,18
best_val_loss,0.32869
epoch,0
loss,0.2923
lr,1e-05
val_loss,0.34477


wandb: wandb version 0.12.15 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


[2022_04_24-17:06:56] Training set: Filtered out 0 of 1291 (0.0%) records of lengths exceeding 510.
[2022_04_24-17:06:56] Validation set: Filtered out 0 of 130 (0.0%) records of lengths exceeding 510.
[2022_04_24-17:06:56] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


...

Epoch 2/100
11/11 [==============================] - 1s 110ms/step - loss: 0.5290 - val_loss: 0.4886
Epoch 3/100
11/11 [==============================] - 1s 107ms/step - loss: 0.4669 - val_loss: 0.4886
Epoch 4/100
11/11 [==============================] - 1s 108ms/step - loss: 0.4450 - val_loss: 0.5069
Epoch 5/100
11/11 [==============================] - 1s 108ms/step - loss: 0.4493 - val_loss: 0.5267
Epoch 6/100
11/11 [==============================] - 1s 109ms/step - loss: 0.4278 - val_loss: 0.5224

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.
Epoch 7/100
11/11 [==============================] - 1s 108ms/step - loss: 0.4439 - val_loss: 0.4984
Epoch 8/100
11/11 [==============================] - 1s 111ms/step - loss: 0.4117 - val_loss: 0.4788
Epoch 9/100
11/11 [==============================] - 1s 107ms/step - loss: 0.4132 - val_loss: 0.4930
Epoch 10/100
11/11 [==============================] - 1s 111ms/step - loss: 0.4049 - val_loss: 0.4924
Epoch 11/

11/11 [==============================] - 10s 422ms/step - loss: 0.3829 - val_loss: 0.4600
Epoch 2/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3655 - val_loss: 0.4604
Epoch 3/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3596 - val_loss: 0.4616
Epoch 4/100
11/11 [==============================] - 3s 248ms/step - loss: 0.3575 - val_loss: 0.4787
Epoch 5/100
11/11 [==============================] - 3s 248ms/step - loss: 0.3616 - val_loss: 0.4599
Epoch 6/100
11/11 [==============================] - 3s 248ms/step - loss: 0.3555 - val_loss: 0.4684
Epoch 7/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3428 - val_loss: 0.4660
Epoch 8/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3364 - val_loss: 0.4654
Epoch 9/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3249 - val_loss: 0.4736

Epoch 00009: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
Epoch 10/100
11/11 [=

21/21 [==============================] - 13s 362ms/step - loss: 0.3510 - val_loss: 0.4543


INFO:tensorflow:Assets written to: ../../data/protein_bert/by_data_size/2022_04_22_size1/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/by_data_size/2022_04_22_size1/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [19]:
f1s

{0.5: 0.37894736842105264,
 0.6: 0.4864864864864865,
 0.7: 0.43010752688172044,
 0.8: 0.4807692307692308,
 0.9: 0.5094339622641509,
 1: 0.5531914893617021}

In [34]:
train_set

,Antibody_ID,heavy,light,Y,cluster,seq
535,2g60,EVQLQQSGGELAKPGASVKMSCKSSGYTFTAYAIHWAKQAAGAGLE...,DVLMTQAPLTLPVSLGDQASISCRSSQAIVHANGNTYLEWYLQKPG...,0,911,EVQLQQSGGELAKPGASVKMSCKSSGYTFTAYAIHWAKQAAGAGLE...
455,2a1w,DVKLVESGGGLVKPGGSLRLSCAASGFTFRNYGMSWVRQTPEKRLE...,DVLMTQSPLSLPVSLGDQASISCRCSQSIVKSNGHTYLEWYLQKPG...,0,723,DVKLVESGGGLVKPGGSLRLSCAASGFTFRNYGMSWVRQTPEKRLE...
459,2a77,DVKLVESGGGLVKPGGSLRLSCAASGFTFRNYGMSWVRQTPEKRLE...,DVLMTQSPLSLPVSLGDQASISCRCSQSIVKSNGHTYLEWYLQKPG...,0,723,DVKLVESGGGLVKPGGSLRLSCAASGFTFRNYGMSWVRQTPEKRLE...
1120,4ffy,QVQLLQPGAELVKPGASMKLSCKASGYTFTNWWMHWVRLRPGRGLE...,NIVLTQSPASLAVSLGQRATISCRASESVDHYGNSFIYWYQQKPGQ...,0,478,QVQLLQPGAELVKPGASMKLSCKASGYTFTNWWMHWVRLRPGRGLE...
851,3l5x,EVTLKESGPVLVKPTETLTLTCTVSGFSLSTYGMGVGWIRQPPGKA...,EIVLTQSPATLSLSPGERATLSCRASKSISKYLAWYQQKPGQAPRL...,0,433,EVTLKESGPVLVKPTETLTLTCTVSGFSLSTYGMGVGWIRQPPGKA...
...,...,...,...,...,...,...
1664,5f9w,QVQLVQSGAEVKKPGASVTVSCQASGYTFTNYYVHWVRQAPGQGLQ...,EIVLTQSPATLSVSPGERATLSCRASQSVRSNLAWYQQRPGQAPRL...,0,271,QVQLVQSGAEVKKPGASVTVSCQASGYTFTNYYVHWVRQAPGQGLQ...
2017,5x5x,QVKLQQSGAEFVKAGASVKLSCKTSGYTFNNYWIHWVKQSPGQGLE...,DIELTQSPLSLPVSLGDQASISCTSSQSLLHSNGDTYLHWYLQKPG...,0,861,QVKLQQSGAEFVKAGASVKLSCKTSGYTFNNYWIHWVKQSPGQGLE...
1400,4qww,EVQLVESGGGLVQPKGSLKLSCAASGFTFNTYAMHWVRQAPGKGLE...,QIVLTQSPAIMSASPGEKVTMTCSASSSVSYMYWYHQKPGSSPKPW...,0,436,EVQLVESGGGLVQPKGSLKLSCAASGFTFNTYAMHWVRQAPGKGLE...
59,1cgs,RVQLLESGAELMKPGASVQISCKATGYTFSEYWIEWVKERPGHGLE...,ELVMTQSPLSLPVSLGDQASISCRPSQSLVHSNGNTYLHWYLQKPG...,0,103,RVQLLESGAELMKPGASVQISCKATGYTFSEYWIEWVKERPGHGLE...


# CV on all data

In [26]:
chen_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_data_w_clusters.csv"), index_col=0)
chen_data.head()

,Antibody_ID,heavy,light,Y,cluster
0,12e8,EVQLQQSGAEVVRSGASVKLSCTASGFNIKDYYIHWVKQRPEKGLE...,DIVMTQSQKFMSTSVGDRVSITCKASQNVGTAVAWYQQKPGQSPKL...,0,677
1,15c8,EVQLQQSGAELVKPGASVKLSCTASGFNIKDTYMHWVKQKPEQGLE...,DIVLTQSPAIMSASLGERVTMTCTASSSVSSSNLHWYQQKPGSSPK...,0,685
2,1a0q,EVQLQESDAELVKPGASVKISCKASGYTFTDHVIHWVKQKPEQGLE...,DIELTQSPSSLSASLGGKVTITCKASQDIKKYIGWYQHKPGKQPRL...,1,102
3,1a14,QVQLQQSGAELVKPGASVRMSCKASGYTFTNYNMYWVKQSPGQGLE...,DIELTQTTSSLSASLGDRVTISCRASQDISNYLNWYQQNPDGTVKL...,0,442
4,1a2y,QVQLQESGPGLVAPSQSLSITCTVSGFSLTGYGVNWVRQPPGKGLE...,DIVLTQSPASLSASVGETVTITCRASGNIHNYLAWYQQKQGKSPQL...,0,59


In [43]:
chen_data["cluster"].value_counts()

18     59
24     35
28     28
8      25
7      21
       ..
588     1
562     1
786     1
722     1
329     1
Name: cluster, Length: 932, dtype: int64

In [56]:
def split_into_k_sets(k, data):
    total = len(data)
    size = total // k + 1
    clusters_by_size = data["cluster"].value_counts().index
    cluster_sizes = data["cluster"].value_counts()
    groups = { i: [] for i in range(k) }
    group_nums = { i: [] for i in range(k) }
    group = 0
    for clust in clusters_by_size:
        start_group = group
        if len(groups[group]) + cluster_sizes[clust] > size:
            group += 1
            group = group % k
        while len(groups[group]) + cluster_sizes[clust] > size and group != start_group:
            group += 1
            group = group % k
        if len(groups[group]) < size:
            groups[group] += list(data[data["cluster"] == clust].index)
            group_nums[group].append(clust)
        else:
            print("error")
    return groups, group_nums

In [57]:
indices, clusters = split_into_k_sets(10, chen_data)
for key, gr in indices.items():
    print(len(gr))

152
154
155
154
156
156
156
156
156
156


In [46]:
chen_data["seq"] = chen_data["heavy"] + chen_data["light"]

In [20]:
def train_and_save_named_model(train_data, valid_data, test_data, name, project_name):
    wandb.init(project=project_name, entity="kvetab")
    model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
            get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

    training_callbacks = [
        keras.callbacks.ReduceLROnPlateau(patience = patience[1], factor = 0.25, min_lr = 1e-07, verbose = 1),
        keras.callbacks.EarlyStopping(patience = patience[0], restore_best_weights = True),
        WandbCallback()
    ]

    epoch_num = 100
    batch_size = 128
    #learning_rate = 1e-5
    wandb.config = {
          "learning_rate": learning_rate,
          "epochs": epoch_num * 2,
          "batch_size": batch_size
        }
    finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data["seq"], train_data["Y"], valid_data['seq'], valid_data["Y"], \
            seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = learning_rate, begin_with_frozen_pretrained_layers = True, \
            lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = learning_rate / 10, callbacks = training_callbacks)
    mod = model_generator.create_model(seq_len = 512)
    mod.save(path.join(DATA_DIR, f"protein_bert/{name}"))

    results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
            start_seq_len = 512, start_batch_size = 32)
    fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
    f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
    return confusion_matrix, f1

In [71]:
cms = {}
f1s = {}

In [75]:
for i in range(10):
    test = chen_data.loc[indices[i]]
    remaining = [idx for idx in list(chen_data.index) if idx not in indices[i]]
    train = chen_data.loc[remaining]
    train, valid = train_test_split(train, test_size=0.1, random_state=333)
    cm, f1 = train_and_save_named_model(train, valid, test, f"10-fold-cv/2022_04_24_split_{i}", "10_fold_cv")
    cms[i] = cm
    f1s[i] = f1

epoch,▁▂▂▃▃▄▅▅▆▆▇▇█▁▂▂▃▃▄▅▅▆▁
loss,▃▇█▆▂▃▄▁▁▃▃▂▂▁▁▁▁▁▁▁▁▁▁
lr,███████████▃▃▁▁▁▁▁▁▁▁▁▁
val_loss,▇█▆▂▂▃▁▂▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁
best_epoch,2
best_val_loss,0.5482
epoch,0
loss,0.47977
lr,1e-05
val_loss,0.55076


wandb: wandb version 0.12.15 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


[2022_04_25-08:53:30] Training set: Filtered out 0 of 1259 (0.0%) records of lengths exceeding 510.
[2022_04_25-08:53:30] Validation set: Filtered out 0 of 140 (0.0%) records of lengths exceeding 510.
[2022_04_25-08:53:30] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
10/10 [==============================] - 10s 310ms/step - loss: 0.8610 - val_loss: 0.6941


...

Epoch 2/100
10/10 [==============================] - 1s 113ms/step - loss: 0.6446 - val_loss: 0.4759
Epoch 3/100
10/10 [==============================] - 1s 115ms/step - loss: 0.5166 - val_loss: 0.4526
Epoch 4/100
10/10 [==============================] - 1s 113ms/step - loss: 0.4499 - val_loss: 0.4427
Epoch 5/100
10/10 [==============================] - 1s 113ms/step - loss: 0.4295 - val_loss: 0.4454
Epoch 6/100
10/10 [==============================] - 1s 111ms/step - loss: 0.4271 - val_loss: 0.4376
Epoch 7/100
10/10 [==============================] - 1s 112ms/step - loss: 0.4161 - val_loss: 0.4305
Epoch 8/100
10/10 [==============================] - 1s 109ms/step - loss: 0.4028 - val_loss: 0.4279
Epoch 9/100
10/10 [==============================] - 1s 110ms/step - loss: 0.4039 - val_loss: 0.4305
Epoch 10/100
10/10 [==============================] - 1s 108ms/step - loss: 0.3937 - val_loss: 0.4262
Epoch 11/100
10/10 [==============================] - 1s 108ms/step - loss: 0.3885 - val_l

10/10 [==============================] - 11s 466ms/step - loss: 0.3588 - val_loss: 0.4069
Epoch 2/100
10/10 [==============================] - 3s 267ms/step - loss: 0.3572 - val_loss: 0.4109
Epoch 3/100
10/10 [==============================] - 3s 268ms/step - loss: 0.3392 - val_loss: 0.4149
Epoch 4/100
10/10 [==============================] - 3s 268ms/step - loss: 0.3302 - val_loss: 0.4168
Epoch 5/100
10/10 [==============================] - 3s 268ms/step - loss: 0.3467 - val_loss: 0.4003
Epoch 6/100
10/10 [==============================] - 3s 271ms/step - loss: 0.3347 - val_loss: 0.4144
Epoch 7/100
10/10 [==============================] - 3s 268ms/step - loss: 0.3256 - val_loss: 0.4085
Epoch 8/100
10/10 [==============================] - 3s 269ms/step - loss: 0.3112 - val_loss: 0.3991
Epoch 9/100
10/10 [==============================] - 3s 268ms/step - loss: 0.3152 - val_loss: 0.4014
Epoch 10/100
10/10 [==============================] - 3s 271ms/step - loss: 0.3037 - val_loss: 0.3940


20/20 [==============================] - 12s 346ms/step - loss: 0.3001 - val_loss: 0.3876


INFO:tensorflow:Assets written to: ../../data/protein_bert/10-fold-cv/2022_04_24_split_9/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/10-fold-cv/2022_04_24_split_9/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [76]:
f1s

{0: 0.44155844155844154,
 1: 0.5806451612903226,
 2: 0.5714285714285714,
 3: 0.4897959183673469,
 4: 0.41509433962264153,
 5: 0.5714285714285714,
 6: 0.5666666666666667,
 7: 0.42857142857142855,
 8: 0.38596491228070173,
 9: 0.4642857142857143}

In [77]:
from statistics import mean

In [82]:
f1_scores = [value for key, value in f1s.items()]
mean(f1_scores)

0.4915439725500407

In [81]:
from statistics import stdev

In [84]:
stdev(f1_scores)

0.07494233825267892

wandb: Network error (ReadTimeout), entering retry loop.


In [96]:
seq_len = 512

In [100]:
for i in range(10):
    test = chen_data.loc[indices[i]]
    encoded_test_set = encode_dataset(test["seq"], test["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
        dataset_name = 'Tap set')
    test_X, test_Y, test_sample_weigths = encoded_test_set
    filename = path.join(DATA_DIR, f"evaluations/protein_bert/10-fold-cv/2022_04_24_split_{i}.csv")
    model_path = path.join(DATA_DIR, f"protein_bert/10-fold-cv/2022_04_24_split_{i}")
    model = keras.models.load_model(model_path)
    y_pred = model.predict(test_X, batch_size=32)
    y_pred_classes = (y_pred >= 0.5)
    str_preds = [str(int(pred)) for pred in y_pred_classes]
    with open(filename, "wt") as f:
        f.write(",".join(str_preds) + "\n")
    metric_dict = {
        "f1": float(metrics.f1_score(test_Y, y_pred_classes)),
        "acc": float(metrics.accuracy_score(test_Y, y_pred_classes)),
        "mcc": float(metrics.matthews_corrcoef(test_Y, y_pred_classes)),
        "auc": float(metrics.roc_auc_score(test_Y, y_pred_classes)),
        "precision": float(metrics.precision_score(test_Y, y_pred_classes)),
        "recall": float(metrics.recall_score(test_Y, y_pred_classes))
    }
    filename_sum = path.join(DATA_DIR, f"evaluations/protein_bert/10-fold-cv/all.csv")
    line = [f"10-fold-cv/2022_04_24_split_{i}", metric_dict["f1"], metric_dict["mcc"], metric_dict["acc"],metric_dict["precision"],metric_dict["recall"],metric_dict["auc"]]
    with open(filename_sum, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter='\t')
        csvwriter.writerow(line)

[2022_04_25-11:05:07] Tap set: Filtered out 0 of 152 (0.0%) records of lengths exceeding 510.
[2022_04_25-11:05:15] Tap set: Filtered out 0 of 154 (0.0%) records of lengths exceeding 510.
[2022_04_25-11:05:23] Tap set: Filtered out 0 of 155 (0.0%) records of lengths exceeding 510.
[2022_04_25-11:05:32] Tap set: Filtered out 0 of 154 (0.0%) records of lengths exceeding 510.
[2022_04_25-11:05:40] Tap set: Filtered out 0 of 156 (0.0%) records of lengths exceeding 510.
[2022_04_25-11:05:49] Tap set: Filtered out 0 of 156 (0.0%) records of lengths exceeding 510.
[2022_04_25-11:05:57] Tap set: Filtered out 0 of 156 (0.0%) records of lengths exceeding 510.
[2022_04_25-11:06:06] Tap set: Filtered out 0 of 156 (0.0%) records of lengths exceeding 510.
[2022_04_25-11:06:14] Tap set: Filtered out 0 of 156 (0.0%) records of lengths exceeding 510.
[2022_04_25-11:06:23] Tap set: Filtered out 0 of 156 (0.0%) records of lengths exceeding 510.


In [102]:
test = chen_data.loc[indices[0]]
test.to_csv(path.join(DATA_DIR, "evaluations/comparison/y_true_0.csv"))

# Comparison with best ML

In [92]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import pickle
import json
import csv

In [86]:
def logistic_regression(preprocessing, data_name, hp_dir):
    filename = path.join(hp_dir, f"logistic_regression_{data_name}_{preprocessing}.json")
    parameters = json.load(open(filename))
    #C = float(parameters["C"])
    lr = LogisticRegression(
        class_weight='balanced', max_iter=1000, random_state=42,
        C=float(parameters["C"]), penalty=parameters["penalty"], solver=parameters["solver"]
    )
    return lr, parameters, "logistic_regression"

In [87]:
def pybiomed(train_df, test_df, tap_df):
    x_chen = pd.read_feather(path.join(DATA_DIR, "chen/pybiomed/X_data.ftr"))
    x_chen_train = x_chen.merge(train_df[["Antibody_ID", "Y"]].reset_index(), left_on="Ab_ID", right_on="Antibody_ID").set_index('index').drop("Antibody_ID", axis=1)
    x_chen_test = x_chen.merge(test_df[["Antibody_ID", "Y"]].reset_index(), left_on="Ab_ID", right_on="Antibody_ID").set_index('index').drop("Antibody_ID", axis=1)
    x_tap = pd.read_feather(path.join(DATA_DIR, "tap/pybiomed/X_TAP_data.ftr"))
    x_tap = x_tap.loc[tap_df.index]
    return x_chen_train, x_chen_test, x_tap

def bert(train_df, test_df, tap_df):
    x_chen = pd.read_feather(path.join(DATA_DIR, "chen/embeddings/bert/bert_chen_embeddings.ftr"))
    x_chen_train = x_chen.merge(train_df[["Antibody_ID", "Y"]].reset_index(), left_on="Ab_ID", right_on="Antibody_ID").set_index('index').drop("Antibody_ID", axis=1)
    x_chen_test = x_chen.merge(test_df[["Antibody_ID", "Y"]].reset_index(), left_on="Ab_ID", right_on="Antibody_ID").set_index('index').drop("Antibody_ID", axis=1)
    x_tap = pd.read_feather(path.join(DATA_DIR, "tap/embeddings/bert/bert_tap_embeddings.ftr"))
    x_tap = x_tap.drop("Ab_ID", axis=1)
    x_tap = x_tap.loc[tap_df.index]
    return x_chen_train, x_chen_test, x_tap

In [88]:
def scaling(train_df, test_df, tap_df):
    scaler = StandardScaler()
    scaler.fit(train_df.drop(["Ab_ID", "Y"], axis=1))
    x_train_tr = scaler.transform(train_df.drop(["Ab_ID", "Y"], axis=1))
    x_train_df = pd.DataFrame(data=train_df,  index=train_df.index, columns=train_df.drop(["Ab_ID", "Y"], axis=1).columns)
    x_train_df["Ab_ID"] = train_df["Ab_ID"]
    
    x_test_tr = scaler.transform(test_df.drop(["Ab_ID", "Y"], axis=1))
    x_test_df = pd.DataFrame(data=test_df,  index=test_df.index, columns=test_df.drop(["Ab_ID", "Y"], axis=1).columns)
    x_test_df["Y"] = test_df["Y"]
    x_test_df["Ab_ID"] = test_df["Ab_ID"]
    
    x_tap_tr = scaler.transform(tap_df)
    x_tap_df = pd.DataFrame(data=tap_df,  index=tap_df.index, columns=tap_df.columns)

    return x_train_df, train_df["Y"], x_test_df, x_tap_df

In [90]:
def train_and_eval(model_name, classifier, X_train, y_train, X_valid, y_valid):
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_valid)
    filename = path.join(DATA_DIR, "evaluations/comparison", "models", f"{model_name}.pkl")
    with open(filename, 'wb') as f:
        pickle.dump(classifier, f)
    filename = path.join(DATA_DIR, "evaluations/comparison", f"{model_name}.csv")
    str_preds = [str(int(pred)) for pred in y_pred]
    with open(filename, "wt") as f:
        f.write(",".join(str_preds) + "\n")
    metric_dict = {
        "f1": float(metrics.f1_score(y_valid, y_pred)),
        "acc": float(metrics.accuracy_score(y_valid, y_pred)),
        "mcc": float(metrics.matthews_corrcoef(y_valid, y_pred)),
        "auc": float(metrics.roc_auc_score(y_valid, y_pred)),
        "precision": float(metrics.precision_score(y_valid, y_pred)),
        "recall": float(metrics.recall_score(y_valid, y_pred))
    }
    filename_sum = os.path.join(DATA_DIR, f"evaluations/comparison/all.csv")
    line = [model_name, metric_dict["f1"], metric_dict["mcc"], metric_dict["acc"],metric_dict["precision"],metric_dict["recall"],metric_dict["auc"]]
    with open(filename_sum, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter='\t')
        csvwriter.writerow(line)

In [93]:
tap_data = pd.read_csv(path.join(DATA_DIR, "tap/tap_not_in_chen.csv"))
for i in range(10):
    test = chen_data.loc[indices[i]]
    remaining = [idx for idx in list(chen_data.index) if idx not in indices[i]]
    train = chen_data.loc[remaining]
    x_train, x_test, x_tap = pybiomed(train, test, tap_data)
    x_train_tr, y_train_tr, x_test_tr, tap_tr = scaling(x_train, x_test, x_tap)
    classifier, params, model_label = logistic_regression("scaling", "pybiomed", path.join(DATA_DIR, "evaluations/hyperparameters"))
    train_and_eval(f"logistic_regression_pybiomed_{i}", classifier, x_train_tr.drop(["Ab_ID"], axis=1), 
                    y_train_tr, x_test_tr.drop(["Ab_ID", "Y"], axis=1), x_test_tr["Y"])

In [94]:
tap_data = pd.read_csv(path.join(DATA_DIR, "tap/tap_not_in_chen.csv"))
for i in range(10):
    test = chen_data.loc[indices[i]]
    remaining = [idx for idx in list(chen_data.index) if idx not in indices[i]]
    train = chen_data.loc[remaining]
    x_train, x_test, x_tap = bert(train, test, tap_data)
    x_train_tr, y_train_tr, x_test_tr, tap_tr = scaling(x_train, x_test, x_tap)
    classifier, params, model_label = logistic_regression("scaling", "bert", path.join(DATA_DIR, "evaluations/hyperparameters"))
    train_and_eval(f"logistic_regression_bert_{i}", classifier, x_train_tr.drop(["Ab_ID"], axis=1), 
                    y_train_tr, x_test_tr.drop(["Ab_ID", "Y"], axis=1), x_test_tr["Y"])